In [42]:
!pip install SimpleITK
import SimpleITK as sitk
import numpy as np
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd

Mount Google Drive and load data

In [2]:
# Init pytorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
# DATA PATHS
datasetPath = 'AlzheimersTrainingSet/'
outputPath = 'AD_Model/'
if not os.path.exists(outputPath):
  os.makedirs(outputPath)
# Image format extension:
extensionImages = 'mhd'
tagInPhase = '_I'
tagLabels = '_labels'

In [5]:
# GET DATA
# Check if the data is already in your folder. If it is not, download it
!ls
if os.path.exists(datasetPath):
   !ls
else:
   !gdown --no-check-certificate --folder https://drive.google.com/drive/folders/1WU2PqUBT-jRhykVkTbhTPCQJJDDQUKQJ?usp=sharing

AD_Model  drive  sample_data
Retrieving folder list
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
Processing file 1njRRp17zcRj7h_XQcRats1ceZZ1xT4XT 3DImageAxialPET.nii
Processing file 1vBHcJQIIuy9tCFTsmEf82cMM9Ld_4N2W 3DImageAxialT1.nii
Processing file 1yx0mQtFV2OZWD2jaC3-OIrLUC2mSCR9t 3DImageCoronalPET.nii
Processing file 1y0NOcmzf9qmlRKakYBhg65_bX5ZMWEPI 3DImageCoronalT1.nii
Processing file 1WwPWhFujFTPzY13HrFqRdGQMrSez3KA_ 3DImageSagittalPET.nii
Processing file 1idHTtrSlebsWNpWUx7nFev3csP9oCt6t 3DImageSagittalT1.nii
Processing file 1T62ky9uTs2vpnwu3tPvzf9j69cM_EB80 labels.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
/usr/local/lib/python3.10/dist-packages/urllib3

In [14]:
# READ AND PREPARE TRAINING SET
# Read data set which is already split into training set and validation set
sitkPetImagesDataset = sitk.ReadImage(datasetPath + '3DImageAxialPET.nii')
sitkMriImagesDataset = sitk.ReadImage(datasetPath + '3DImageAxialT1.nii')

petImagesDataset = sitk.GetArrayFromImage(sitkPetImagesDataset)
mriImagesDataset = sitk.GetArrayFromImage(sitkMriImagesDataset)






FileNotFoundError: ignored

In [33]:
df = pd.read_csv(datasetPath + "labels.csv")
labels = df.transpose().iloc[1,:].to_numpy()
print(labels)

['CN' 'CN' 'AD' 'CN' 'CN' 'AD' 'CN' 'CN' 'AD' 'CN' 'CN' 'CN' 'AD' 'AD'
 'AD' 'AD' 'CN' 'AD' 'CN' 'AD' 'AD' 'CN' 'CN' 'CN' 'AD' 'CN' 'CN' 'AD'
 'AD' 'AD' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'AD' 'AD' 'AD' 'CN'
 'CN' 'CN' 'CN' 'AD' 'AD' 'CN' 'AD' 'CN' 'AD' 'CN' 'CN' 'AD' 'CN' 'AD'
 'CN' 'CN' 'CN' 'AD' 'AD' 'CN' 'AD' 'AD' 'CN' 'AD' 'CN' 'CN' 'AD' 'CN'
 'CN' 'CN' 'CN' 'AD' 'CN' 'CN' 'CN' 'CN' 'AD' 'AD' 'AD' 'AD' 'AD' 'CN'
 'AD' 'AD' 'AD' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'AD' 'AD' 'CN' 'AD' 'AD'
 'AD' 'AD' 'CN' 'CN' 'AD' 'AD' 'AD' 'AD' 'AD' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'AD' 'AD' 'CN' 'AD' 'AD' 'AD']


In [35]:
# CREATE TRAINING SET
# Chose one
imagesDataSet = np.expand_dims(petImagesDataset, axis=1) # 1) Only PET
imagesDataSet = np.expand_dims(mriImagesDataset, axis=1) # 2) Only MRI
imagesDataSet = np.concatenate((np.expand_dims(mriImagesDataset, axis=1),
                                np.expand_dims(petImagesDataset, axis=1)), axis=1)# 3) PET+MRI

numImages = imagesDataSet.shape[0]
print(numImages)
trainValidSplit = 0.7;


127


In [36]:
# VISUALIZE SOME IMAGES OF THE TRAINING SET

In [43]:
# LOAD A PRETRAINED MODEL
model_ft = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.4MB/s]


NameError: ignored

In [39]:
# RETRAINING
# Number of  batches:
batchSize = 4
devBatchSize = 1
numBatches = np.ceil(trainingSet['input'].shape[0]/batchSize).astype(int)
devNumBatches = np.ceil(devSet['input'].shape[0]/devBatchSize).astype(int)
# Show results every printStep batches:
plotStep_epochs = 1 # math.inf to disable the plot.
numImagesPerRow = batchSize

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if plotStep_epochs != math.inf:
    figGraphs, axs_graphs = plt.subplots(1, 8, figsize=(15, 8))
# Show dev set loss every showDevLossStep batches:
inputsDevSet = torch.from_numpy(devSet['input'])
gtDevSet = torch.from_numpy(devSet['output'])


model_ft.to(device)

for epoch in range(max_epochs):  # loop over the dataset multiple times
    epochNumbers.append(epoch)


    #### TRAINING ####
    model_ft.train(True)
    for i in range(numBatches):
        # add code using segmentation challenge as example

SyntaxError: ignored

In [ ]:
# Load the best model and segment the validation set
#TODO


IndexError: ignored

In [ ]:
# Visualizing output images:
#TODO

drive  MuscleModel  MuscleTrainingSet  sample_data
